In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, MultiHeadAttention, LayerNormalization

# Generate synthetic data (e.g., exponential decay)
def simulate_ode(k=0.1, y0=1.0, t_max=10, num_points=1000):
    t = np.linspace(0, t_max, num_points)
    y = y0 * np.exp(-k * t)
    return t, y

# Create synthetic data
t, y = simulate_ode()
X = t.reshape(-1, 1)  # Time as input
y = y.reshape(-1, 1)  # State as output

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import optuna

def objective(trial, model_type):
    # Common hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    # Define the model based on the type
    model = Sequential()
    
    if model_type == 'rnn':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(SimpleRNN(units, activation='tanh', return_sequences=True if _ < num_layers - 1 else False))
        model.add(Dense(1))

    elif model_type == 'lstm':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(LSTM(units, activation='tanh', return_sequences=True if _ < num_layers - 1 else False))
        model.add(Dense(1))
        
    elif model_type == 'transformer':
        num_heads = trial.suggest_int('num_heads', 2, 8)
        key_dim = trial.suggest_int('key_dim', 16, 64, step=16)
        model.add(MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, input_shape=(None, 1)))
        model.add(LayerNormalization())
        model.add(Dense(1))

    elif model_type == 'neural_net':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(Dense(units, activation='relu'))
        model.add(Dense(1))

    elif model_type == 'ode':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        model.add(Dense(units, activation='relu', input_dim=1))
        for _ in range(num_layers):
            model.add(Dense(units, activation='relu'))
        model.add(Dense(1))  # Approximate derivative

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    # Train the model
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=10,
        batch_size=batch_size,
        verbose=0,
    )

    # Return the validation loss for Optuna to minimize
    val_loss = history.history['val_loss'][-1]
    return val_loss


In [ ]:
study_rnn = optuna.create_study(direction='minimize')
study_rnn.optimize(lambda trial: objective(trial, model_type='rnn'), n_trials=20)
print("Best RNN parameters:", study_rnn.best_params)


In [ ]:
study_lstm = optuna.create_study(direction='minimize')
study_lstm.optimize(lambda trial: objective(trial, model_type='lstm'), n_trials=20)
print("Best LSTM parameters:", study_lstm.best_params)


In [ ]:
study_transformer = optuna.create_study(direction='minimize')
study_transformer.optimize(lambda trial: objective(trial, model_type='transformer'), n_trials=20)
print("Best Transformer parameters:", study_transformer.best_params)


In [ ]:
study_nn = optuna.create_study(direction='minimize')
study_nn.optimize(lambda trial: objective(trial, model_type='neural_net'), n_trials=20)
print("Best Neural Network parameters:", study_nn.best_params)


In [ ]:
study_ode = optuna.create_study(direction='minimize')
study_ode.optimize(lambda trial: objective(trial, model_type='ode'), n_trials=20)
print("Best ODE parameters:", study_ode.best_params)


In [ ]:
# Example: Using the best RNN model
best_rnn_params = study_rnn.best_params
print("Best RNN Parameters:", best_rnn_params)

Here’s how to use this function to print the architecture for each model:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, MultiHeadAttention, LayerNormalization

def build_best_model(best_params, model_type):
    model = Sequential()
    
    if model_type == 'rnn':
        for _ in range(best_params['num_layers']):
            model.add(SimpleRNN(
                units=best_params['units'],
                activation='tanh',
                return_sequences=True if _ < best_params['num_layers'] - 1 else False
            ))
        model.add(Dense(1))

    elif model_type == 'lstm':
        for _ in range(best_params['num_layers']):
            model.add(LSTM(
                units=best_params['units'],
                activation='tanh',
                return_sequences=True if _ < best_params['num_layers'] - 1 else False
            ))
        model.add(Dense(1))

    elif model_type == 'transformer':
        model.add(MultiHeadAttention(
            num_heads=best_params['num_heads'],
            key_dim=best_params['key_dim'],
            input_shape=(None, 1)
        ))
        model.add(LayerNormalization())
        model.add(Dense(1))

    elif model_type == 'neural_net':
        for _ in range(best_params['num_layers']):
            model.add(Dense(
                units=best_params['units'],
                activation='relu'
            ))
        model.add(Dense(1))

    elif model_type == 'ode':
        model.add(Dense(
            best_params['units'], activation='relu', input_dim=1
        ))
        for _ in range(best_params['num_layers']):
            model.add(Dense(
                best_params['units'], activation='relu'
            ))
        model.add(Dense(1))  # Approximate derivative

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']), loss='mse')
    return model


In [ ]:
3. Output Explanation
Each summary() call will display:
The layers of the model.
The number of parameters in each layer.
The total trainable and non-trainable parameters.


In [ ]:
# Example: Print the best RNN architecture
best_rnn_model = build_best_model(study_rnn.best_params, 'rnn')
print("Best RNN Architecture:")
best_rnn_model.summary()

# Example: Print the best LSTM architecture
best_lstm_model = build_best_model(study_lstm.best_params, 'lstm')
print("\nBest LSTM Architecture:")
best_lstm_model.summary()

# Example: Print the best Transformer architecture
best_transformer_model = build_best_model(study_transformer.best_params, 'transformer')
print("\nBest Transformer Architecture:")
best_transformer_model.summary()

# Example: Print the best Neural Network architecture
best_nn_model = build_best_model(study_nn.best_params, 'neural_net')
print("\nBest Neural Network Architecture:")
best_nn_model.summary()

# Example: Print the best ODE architecture
best_ode_model = build_best_model(study_ode.best_params, 'ode')
print("\nBest ODE Architecture:")
best_ode_model.summary()
